In [ ]:
!pip install unsloth xformers trl peft accelerate bitsandbytes faiss-cpu langchain-community langchain-huggingface sentence-transformers

# RAG

In [ ]:
import gdown

# Скачивание и загрузка клинических рекомендаций
guidelines_id = "1Ctvi5eS39zYDY5paXmIgV9PnVaiRq1UU"
guidelines_name = "russco"
gdown.download(id=guidelines_id, output="guidelines.zip", quiet=False)
!unzip -O CP866 -o -q guidelines.zip -d guidelines

In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Настройка заголовков для разделения
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# Инициализируем сплиттеры
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Загрузка документов
loader = DirectoryLoader("./guidelines/"+guidelines_name, glob="./*.md", loader_cls=TextLoader)
documents = loader.load()

final_chunks = []
for doc in documents:
    file_source = doc.metadata.get('source', 'unknown')
    file_name = "".join(os.path.basename(file_source).split(".")[:-1])

    header_splits = markdown_splitter.split_text(doc.page_content)
    for header_chunk in header_splits:
        header_chunk.metadata['source'] = file_name
        sub_chunks = text_splitter.split_documents([header_chunk])
        final_chunks.extend(sub_chunks)

# Загрузка модели эмбеддингов
model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 3. Создание векторной базы
vector_db = FAISS.from_documents(final_chunks, embeddings)
vector_db.save_local("russco_faiss")


# Fine-tuning

In [ ]:
import json
import random
import gdown

# Скачивание и загрузка датасета
dataset_id = "1PY_woE3FPjrZcVDKcShVmdCFyBpv_s60"
gdown.download(id=dataset_id, output="full_dset.json", quiet=False)

with open("./full_dset.json", "r") as file:
    data_list = json.load(file)

# Разделение датасета на выборки
random.seed(42)
random.shuffle(data_list)
n_test = round(0.1*len(data_list))
data_list_test, data_list_train = data_list[0:n_test], data_list[n_test:]

# Сохранение выборок
with open("./train_dset.json", 'w', encoding='utf-8') as json_file:
    json.dump(data_list_train, json_file, indent=4, ensure_ascii=False)
with open("./test_dset.json", 'w', encoding='utf-8') as json_file:
    json.dump(data_list_test, json_file, indent=4, ensure_ascii=False)

print(f"\n\nЧисло записей: {len(data_list)}\nРазмер обучающей выборки: {len(data_list_train)}\nРазмер тестовой выборки: {len(data_list_test)}\n")
for key, value in data_list[1].items():
  print(f"===== {key} =====\n{value}\n")

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

# Загрузка языковой модели
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

# Добавление LoRA адаптеров
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Степень адаптации (чем выше, тем умнее, но тяжелее)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state=42
)

In [ ]:
from datasets import Dataset

# Формирование промптов для дообучения

prompt_style = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.

### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
{}"""

def formatting_prompts_func(examples):
    inputs = examples["treatment_plan"]
    context = examples["clinical_guidelines"]
    outputs = examples["corrections"]
    texts = []
    for i, c, o in zip(inputs, context, outputs):
        text = prompt_style.format(i, c, o) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = Dataset.from_list(data_list_train)
dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset[0]['text'])

In [ ]:
import pandas as pd

# Расчет статистик по размеру запросов

def count_tokens(example):
    return {"token_count": len(tokenizer.encode(example["text"]))}

dataset_with_counts = dataset.map(count_tokens)
df = dataset_with_counts.to_pandas()

print(f"Средняя длина: {df['token_count'].mean()}")
print(f"Максимальная длина: {df['token_count'].max()}")
print(f"95-й перцентиль (рекомендуемый max_seq_length): {df['token_count'].quantile(0.95)}")

In [ ]:
import gc

# Очистка кэша перед обучением
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Дообучение модели
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)
trainer.train()

In [ ]:
# Сохранение адаптеров
model.save_pretrained("medical_lora_model")
tokenizer.save_pretrained("medical_lora_model")

# Testing

In [ ]:
import torch
from unsloth import FastLanguageModel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "medical_lora_model",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

model_name = "intfloat/multilingual-e5-small"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = FAISS.load_local("russco_faiss", embeddings, allow_dangerous_deserialization=True)

def get_context(treatment_plan):
    search_query = f"query: {treatment_plan}"
    docs = db.similarity_search(search_query, k=2)

    context = "\n---\n".join([
        f"Источник ({d.metadata.get('Header 2', 'Общее')}): {d.page_content}"
        for d in docs
    ])
    return context

prompt_template = """Ниже представлен план лечения. Исправь его, опираясь на клинические рекомендации.
{}
### План лечения:
{}

### Рекомендации:
{}

### Исправленния:
"""

def correct_treatment_plan(treatment_plan, context, doctor=True):
    added_text = "Формулируй ответ простыми словами, которые будут понятны пациенту.\n"
    if doctor:
        added_text = ""
    final_prompt = prompt_template.format(added_text, treatment_plan, context)
    inputs = tokenizer([final_prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    result = tokenizer.batch_decode(outputs, skip_special_tokens = True)
    corrections = result[0][len(final_prompt):]
    return corrections

In [ ]:
treatment_plan = """
Диагноз: рак молочной железы.
Для определения тактики лечения проводится комплексное исследование опухоли:
ИГХ-профиль: Оценка рецепторов к гормонам (эстрогену и прогестерону), статуса HER2 и индекса пролиферации Ki-67.
Генетическое тестирование:
Поиск мутаций BRCA1/2 (обязательно при семейной истории болезни или специфических формах рака).
Анализ PIK3CA (необходим при метастатическом гормонозависимом раке для подбора таргетной терапии).
Определение уровня PD-L1 (критично для выбора иммунотерапии при тройном негативном подтипе).
Алгоритмы медикаментозного лечения
1. Профилактическое (адъювантное) лечение:
HER2-положительный тип: Стандарт — трастузумаб в течение года. При поражении лимфоузлов добавляется пертузумаб. Если после предоперационной химии осталась опухоль, назначается Кадсила (T-DM1).
Тройной негативный тип: При остаточной опухоли применяется капецитабин. Пациентам с мутацией BRCA показан олапариб.
2. Терапия при распространенных (метастатических) формах:
Гормонозависимый HER2-отрицательный: Основной стандарт первой линии — комбинация ингибиторов CDK4/6 (рибо-, палбо- или абемациклиб) с гормональной терапией. При наличии мутации PIK3CA к фулвестранту добавляют алпелисиб.
HER2-положительный: Начинают с комбинации таксанов и двойной блокады (трастузумаб + пертузумаб). Во второй линии используют T-DM1, в последующих — инновационный конъюгат T-DXd.
Тройной негативный: При высоком уровне PD-L1 эффективно сочетание химиотерапии с иммунопрепаратом пембролизумабом. Носителям мутации BRCA назначаются таргетные PARP-ингибиторы (олапариб/талазопариб).
"""

context = get_context(treatment_plan)
corrections = correct_treatment_plan(treatment_plan, context)
print(corrections)

In [ ]:
context